In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import classifier
import modelnet

batch_size = 1
downsample = 2    #For 5000 points use 2, for 1000 use 10, for 100 use 100
network_dim = 512  #For 5000 points use 512, for 1000 use 256, for 100 use 256
num_repeats = 10    #Number of times to repeat the experiment
data_path = 'ModelNet40_cloud.h5'

In [2]:
model_fetcher = modelnet.ModelFetcher(data_path, batch_size, downsample, do_standardize=True, do_augmentation=True)

def eval_model(model_name):
    D = classifier.DTanh(network_dim, pool='max1').cuda()
    D.load_state_dict(torch.load(model_name))
    D.eval()
    counts = 0
    sum_acc = 0.0
    batch_results = []
    batch_labels = []
    batch_data = []
    for x, _, y in model_fetcher.test_data():
        counts += len(y)
        X = Variable(torch.cuda.FloatTensor(x))
        Y = Variable(torch.cuda.LongTensor(y))
        f_X = D(X)
        batch_results.append(f_X.detach().cpu().numpy())
        batch_labels.append(y)
        batch_data.append(x)
        sum_acc += (f_X.max(dim=1)[1] == Y).float().sum().data.cpu().item()
        del X,Y,f_X
    test_acc = sum_acc/counts
    results = np.concatenate(batch_results, 0)
    labels = np.concatenate(batch_labels, 0)
    data = np.concatenate(batch_data, 0)
    print('Final Test Accuracy: {0:0.3f}'.format(test_acc))
    print(results.shape)
    return results, labels, data

In [3]:
model_results_list = []
for i in range(1):
    mr, labels, data = eval_model('model_' + str(i + 1) + '.pt')
    model_results_list.append(mr)

DTanh(
  (phi): Sequential(
    (0): PermEqui1_max(
      (Gamma): Linear(in_features=3, out_features=512, bias=True)
    )
    (1): Tanh()
    (2): PermEqui1_max(
      (Gamma): Linear(in_features=512, out_features=512, bias=True)
    )
    (3): Tanh()
    (4): PermEqui1_max(
      (Gamma): Linear(in_features=512, out_features=512, bias=True)
    )
    (5): Tanh()
  )
  (ro): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.5)
    (4): Linear(in_features=512, out_features=40, bias=True)
  )
)


Final Test Accuracy: 0.900
(2466, 40)


In [5]:
print(data.shape)
np.save('xdata.npy', data)

(2466, 5000, 3)


In [13]:
model_results = np.stack(model_results_list, -1)
print(model_results.shape, labels.shape)

(2466, 40, 10) (2466,)


In [14]:
np.save('probabilities.npy', model_results)
np.save('true_labels.npy', labels)